In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
from scipy import signal
import os
from scipy.linalg import circulant
import matplotlib.pyplot as plt
import random
import scipy
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
from random import shuffle

In [2]:
file = open('D:\PhD topics\Datasets\Afib_datasets\store_noise_one_mean_final2_-20.pkl', 'rb')
#file= open('store_no_noise.pkl','rb')
obj = pickle.load(file)
file.close()

In [3]:
ecg_train=obj['ecg_train']
ecg_valid=obj['ecg_valid']
ecg_test=obj['ecg_test']
ppg_train=obj['ppg_train']
ppg_valid=obj['ppg_valid']
ppg_test=obj['ppg_test']
y_train=obj['y_train']
y_valid=obj['y_valid']
y_test=obj['y_test']

In [4]:
ecg_sqi_train=obj['ecg_sqi_train']
ppg_sqi_train=obj['ppg_sqi_train']
ecg_sqi_valid=obj['ecg_sqi_valid']
ppg_sqi_valid=obj['ppg_sqi_valid']
ecg_sqi_test=obj['ecg_sqi_test']
ppg_sqi_test=obj['ppg_sqi_test']

In [5]:
del obj

In [6]:
indices=np.isnan(ecg_train).any(axis=1)

In [7]:
ppg_train

array([[-0.06349457,  0.08844781, -0.0326824 , ...,  0.02655176,
         0.09326861,  0.11718857],
       [ 0.01169633,  0.01169633,  0.01169633, ...,  0.01180454,
         0.01180454,  0.01169633],
       [-0.1079712 ,  0.14550357, -0.02788106, ..., -0.12567619,
         0.12003856,  0.11083796],
       ...,
       [-0.246522  ,  0.79480254,  1.42390128, ..., -0.0849805 ,
        -0.69458182,  0.44167311],
       [-0.25915785,  0.18577151, -0.18441675, ...,  0.29162016,
         0.59671575,  0.34884971],
       [ 0.05922131,  0.06896764,  0.07886163, ..., -0.05271372,
        -0.05064632, -0.04976029]])

In [8]:
ecg_sqi_train_mean=np.mean(ecg_sqi_train, axis=0)
ecg_sqi_train_std=np.std(ecg_sqi_train, axis=0)

for i in range(ecg_sqi_train.shape[0]):
    ecg_sqi_train[i,:]=(ecg_sqi_train[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std
    
for i in range(ecg_sqi_valid.shape[0]):
    ecg_sqi_valid[i,:]=(ecg_sqi_valid[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std
    
for i in range(ecg_sqi_test.shape[0]):
    ecg_sqi_test[i,:]=(ecg_sqi_test[i,:]-ecg_sqi_train_mean)/ecg_sqi_train_std

In [9]:
ppg_sqi_train_mean=np.mean(ppg_sqi_train, axis=0)
ppg_sqi_train_std=np.std(ppg_sqi_train, axis=0)

for i in range(ppg_sqi_train.shape[0]):
    ppg_sqi_train[i,:]=(ppg_sqi_train[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std
    
for i in range(ppg_sqi_valid.shape[0]):
    ppg_sqi_valid[i,:]=(ppg_sqi_valid[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std
    
for i in range(ppg_sqi_test.shape[0]):
    ppg_sqi_test[i,:]=(ppg_sqi_test[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std


In [10]:
ecg_train=np.expand_dims(ecg_train, axis=1)
ecg_valid=np.expand_dims(ecg_valid, axis=1)
ppg_train=np.expand_dims(ppg_train, axis=1)
ppg_valid=np.expand_dims(ppg_valid, axis=1)

ecg_sqi_train=np.expand_dims(ecg_sqi_train, axis=1)
ecg_sqi_valid=np.expand_dims(ecg_sqi_valid, axis=1)
ppg_sqi_train=np.expand_dims(ppg_sqi_train, axis=1)
ppg_sqi_valid=np.expand_dims(ppg_sqi_valid, axis=1)

In [11]:
ecg_test=np.expand_dims(ecg_test, axis=1)
ppg_test=np.expand_dims(ppg_test, axis=1)


ecg_sqi_test=np.expand_dims(ecg_sqi_test, axis=1)
ppg_sqi_test=np.expand_dims(ppg_sqi_test, axis=1)


In [12]:
num_classes=2
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [13]:
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = to_categorical(y_test_num, num_classes)

In [14]:
class fusion_model_v3(nn.Module):
    def __init__(self):
        super().__init__()
        
        #ecg alone
        self.bn1_ecg=nn.BatchNorm1d(1)
        self.conv1_ecg = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ecg = nn.LeakyReLU()
        self.max1_ecg = nn.MaxPool1d(2)
        self.drop1_ecg=nn.Dropout(0.25)
        self.conv2_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ecg = nn.MaxPool1d(2)
        self.relu2_ecg = nn.LeakyReLU()
        self.drop2_ecg=nn.Dropout(0.25)
        self.conv3_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ecg=nn.Dropout(0.25)
        self.conv4_ecg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ecg = nn.MaxPool1d(2)
        self.relu4_ecg = nn.LeakyReLU()
        self.bn2_ecg=nn.BatchNorm1d(5)
        self.flat_ecg=nn.Flatten()
        self.drop4_ecg=nn.Dropout(0.25)
        self.linear1_ecg=nn.Linear(660,2)
        self.softm_ecg=nn.Softmax()
        
        #ppg alone
        self.bn1_ppg=nn.BatchNorm1d(1)
        self.conv1_ppg = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ppg = nn.LeakyReLU()
        self.max1_ppg = nn.MaxPool1d(2)
        self.drop1_ppg=nn.Dropout(0.25)
        self.conv2_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ppg = nn.MaxPool1d(2)
        self.relu2_ppg = nn.LeakyReLU()
        self.drop2_ppg=nn.Dropout(0.25)
        self.conv3_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ppg=nn.Dropout(0.25)
        self.conv4_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ppg = nn.MaxPool1d(2)
        self.relu4_ppg = nn.LeakyReLU()
        self.bn2_ppg=nn.BatchNorm1d(5)
        self.flat_ppg=nn.Flatten()
        self.drop4_ppg=nn.Dropout(0.25)
        self.linear1_ppg=nn.Linear(660,2)
        self.softm_ppg=nn.Softmax()
        
        #centre
        self.linear1_centre_ecg=nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=1, padding='same')
        self.linear1_centre_ppg=nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=1, padding='same')
        self.tanh1_ecg = nn.LeakyReLU()
        self.tanh1_ppg = nn.LeakyReLU()
        self.w1_centre=torch.nn.Parameter(torch.tensor(0.5))
        #self.w2_centre=torch.nn.Parameter(torch.tensor(0.5))
        
        self.bn1_centre=nn.BatchNorm1d(1)
        self.conv1_centre = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_centre = nn.Sigmoid()
        
        self.alpha1=torch.nn.Parameter(torch.tensor(0.5))
        
        self.max1_centre = nn.MaxPool1d(2)
        self.drop1_centre=nn.Dropout(0.25)
        self.conv2_centre = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_centre = nn.MaxPool1d(2)
        self.relu2_centre = nn.Sigmoid()
        
        self.alpha2=torch.nn.Parameter(torch.tensor(0.5))
        
        self.drop2_centre=nn.Dropout(0.25)
        self.conv3_centre = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_centre=nn.Dropout(0.25)
        self.conv4_centre = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_centre = nn.MaxPool1d(2)
        self.relu4_centre = nn.Sigmoid()
        
        self.alpha3=torch.nn.Parameter(torch.tensor(0.5))
        
        self.bn2_centre=nn.BatchNorm1d(5)
        self.flat_centre=nn.Flatten()
        
        self.alpha4=torch.nn.Parameter(torch.tensor(0.5))
        
        self.drop4_centre=nn.Dropout(0.25)
        self.linear1_centre=nn.Linear(660,2)
        self.softm_centre=nn.Softmax()
        
        #ecgsqi
        self.bn1_ecgsqi=nn.BatchNorm1d(1)
        self.conv1_ecgsqi = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ecgsqi = nn.LeakyReLU()
        self.max1_ecgsqi = nn.MaxPool1d(2)
        self.drop1_ecgsqi=nn.Dropout(0.25)
        self.conv2_ecgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ecgsqi = nn.MaxPool1d(2)
        self.relu2_ecgsqi = nn.LeakyReLU()
        self.drop2_ecgsqi=nn.Dropout(0.25)
        self.conv3_ecgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ecgsqi=nn.Dropout(0.25)
        self.conv4_ecgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ecgsqi = nn.MaxPool1d(2)
        self.relu4_ecgsqi = nn.LeakyReLU()
        self.bn2_ecgsqi=nn.BatchNorm1d(5)
        self.flat_ecgsqi=nn.Flatten()
        
        #ppgsqi
        self.bn1_ppgsqi=nn.BatchNorm1d(1)
        self.conv1_ppgsqi = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ppgsqi = nn.LeakyReLU()
        self.max1_ppgsqi = nn.MaxPool1d(2)
        self.drop1_ppgsqi=nn.Dropout(0.25)
        self.conv2_ppgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ppgsqi = nn.MaxPool1d(2)
        self.relu2_ppgsqi = nn.LeakyReLU()
        self.drop2_ppgsqi=nn.Dropout(0.25)
        self.conv3_ppgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ppgsqi=nn.Dropout(0.25)
        self.conv4_ppgsqi = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ppgsqi = nn.MaxPool1d(2)
        self.relu4_ppgsqi = nn.LeakyReLU()
        self.bn2_ppgsqi=nn.BatchNorm1d(5)
        self.flat_ppgsqi=nn.Flatten()
        
        
        
        
    
        
    def forward(self, ecg, ppg, ecgsqi, ppgsqi):
        
        ### ecg
        out_ecg_bn1=self.bn1_ecg(ecg)
        out_ecg_conv1= self.conv1_ecg(out_ecg_bn1)
        out_ecg_relu1 = self.relu1_ecg(out_ecg_conv1)
        out_ecg_max1=self.max1_ecg(out_ecg_relu1)
        out_ecg_drop1=self.drop1_ecg(out_ecg_max1)
        out_ecg_conv2=self.conv2_ecg(out_ecg_drop1)
        out_ecg_max2=self.max2_ecg(out_ecg_conv2)
        out_ecg_relu2=self.relu2_ecg(out_ecg_max2)
        out_ecg_drop2=self.drop2_ecg(out_ecg_relu2)
        out_ecg_conv3=self.conv3_ecg(out_ecg_drop2)
        out_ecg_drop3=self.drop3_ecg(out_ecg_conv3)
        out_ecg_conv4=self.conv4_ecg(out_ecg_drop3)
        out_ecg_max4=self.max4_ecg(out_ecg_conv4)
        out_ecg_relu4=self.relu4_ecg(out_ecg_max4)
        out_ecg_bn2=self.bn2_ecg(out_ecg_relu4)
        out_ecg_flat=self.flat_ecg(out_ecg_bn2)
        out_ecg_drop4=self.drop4_ecg(out_ecg_flat)
        out_ecg_linear=self.linear1_ecg(out_ecg_drop4)
        out_ecg_softm=self.softm_ecg(out_ecg_linear)
        
        ### ppg
        out_ppg_bn1=self.bn1_ppg(ppg)
        out_ppg_conv1= self.conv1_ppg(out_ppg_bn1)
        out_ppg_relu1 = self.relu1_ppg(out_ppg_conv1)
        out_ppg_max1=self.max1_ppg(out_ppg_relu1)
        out_ppg_drop1=self.drop1_ppg(out_ppg_max1)
        out_ppg_conv2=self.conv2_ppg(out_ppg_drop1)
        out_ppg_max2=self.max2_ppg(out_ppg_conv2)
        out_ppg_relu2=self.relu2_ppg(out_ppg_max2)
        out_ppg_drop2=self.drop2_ppg(out_ppg_relu2)
        out_ppg_conv3=self.conv3_ppg(out_ppg_drop2)
        out_ppg_drop3=self.drop3_ppg(out_ppg_conv3)
        out_ppg_conv4=self.conv4_ppg(out_ppg_drop3)
        out_ppg_max4=self.max4_ppg(out_ppg_conv4)
        out_ppg_relu4=self.relu4_ppg(out_ppg_max4)
        out_ppg_bn2=self.bn2_ppg(out_ppg_relu4)
        out_ppg_flat=self.flat_ppg(out_ppg_bn2)
        out_ppg_drop4=self.drop4_ppg(out_ppg_flat)
        out_ppg_linear=self.linear1_ppg(out_ppg_drop4)
        out_ppg_softm=self.softm_ppg(out_ppg_linear)
        
        
        ### ecgsqi
        out_ecgsqi_bn1=self.bn1_ecgsqi(ecgsqi)
        out_ecgsqi_conv1= self.conv1_ecgsqi(out_ecgsqi_bn1)
        out_ecgsqi_relu1 = self.relu1_ecgsqi(out_ecgsqi_conv1)
        out_ecgsqi_max1=self.max1_ecgsqi(out_ecgsqi_relu1)
        out_ecgsqi_drop1=self.drop1_ecgsqi(out_ecgsqi_max1)
        out_ecgsqi_conv2=self.conv2_ecgsqi(out_ecgsqi_drop1)
        out_ecgsqi_max2=self.max2_ecgsqi(out_ecgsqi_conv2)
        out_ecgsqi_relu2=self.relu2_ecgsqi(out_ecgsqi_max2)
        out_ecgsqi_drop2=self.drop2_ecgsqi(out_ecgsqi_relu2)
        out_ecgsqi_conv3=self.conv3_ecgsqi(out_ecgsqi_drop2)
        out_ecgsqi_drop3=self.drop3_ecgsqi(out_ecgsqi_conv3)
        out_ecgsqi_conv4=self.conv4_ecgsqi(out_ecgsqi_drop3)
        out_ecgsqi_max4=self.max4_ecgsqi(out_ecgsqi_conv4)
        out_ecgsqi_relu4=self.relu4_ecgsqi(out_ecgsqi_max4)
        out_ecgsqi_bn2=self.bn2_ecgsqi(out_ecgsqi_relu4)
        out_ecgsqi_flat=self.flat_ecgsqi(out_ecgsqi_bn2)

        ### ppgsqi
        out_ppgsqi_bn1=self.bn1_ppgsqi(ppgsqi)
        out_ppgsqi_conv1= self.conv1_ppgsqi(out_ppgsqi_bn1)
        out_ppgsqi_relu1 = self.relu1_ppgsqi(out_ppgsqi_conv1)
        out_ppgsqi_max1=self.max1_ppgsqi(out_ppgsqi_relu1)
        out_ppgsqi_drop1=self.drop1_ppgsqi(out_ppgsqi_max1)
        out_ppgsqi_conv2=self.conv2_ppgsqi(out_ppgsqi_drop1)
        out_ppgsqi_max2=self.max2_ppgsqi(out_ppgsqi_conv2)
        out_ppgsqi_relu2=self.relu2_ppgsqi(out_ppgsqi_max2)
        out_ppgsqi_drop2=self.drop2_ppgsqi(out_ppgsqi_relu2)
        out_ppgsqi_conv3=self.conv3_ppgsqi(out_ppgsqi_drop2)
        out_ppgsqi_drop3=self.drop3_ppgsqi(out_ppgsqi_conv3)
        out_ppgsqi_conv4=self.conv4_ppgsqi(out_ppgsqi_drop3)
        out_ppgsqi_max4=self.max4_ppgsqi(out_ppgsqi_conv4)
        out_ppgsqi_relu4=self.relu4_ppgsqi(out_ppgsqi_max4)
        out_ppgsqi_bn2=self.bn2_ppgsqi(out_ppgsqi_relu4)
        out_ppgsqi_flat=self.flat_ppgsqi(out_ppgsqi_bn2)
        
        ####weighted_sqis
        out_ecgsqi_relu1_w=out_ecgsqi_relu1.div(out_ecgsqi_relu1+out_ppgsqi_relu1+1e-6)
        out_ppgsqi_relu1_w=out_ppgsqi_relu1.div(out_ecgsqi_relu1+out_ppgsqi_relu1+1e-6)
        
        out_ecgsqi_relu2_w=out_ecgsqi_relu2.div(out_ecgsqi_relu2+out_ppgsqi_relu2+1e-6)
        out_ppgsqi_relu2_w=out_ppgsqi_relu2.div(out_ecgsqi_relu2+out_ppgsqi_relu2+1e-6)
        
        out_ecgsqi_relu4_w=out_ecgsqi_relu4.div(out_ecgsqi_relu4+out_ppgsqi_relu4+1e-6)
        out_ppgsqi_relu4_w=out_ppgsqi_relu4.div(out_ecgsqi_relu4+out_ppgsqi_relu4+1e-6)
        
        out_ecgsqi_flat_w=out_ecgsqi_flat.div(out_ecgsqi_flat+out_ppgsqi_flat+1e-6)
        out_ppgsqi_flat_w=out_ppgsqi_flat.div(out_ecgsqi_flat+out_ppgsqi_flat+1e-6)
        
        ### centre
        
        out_centre_ecg=self.linear1_centre_ecg(ecg)
        out_centre_ppg=self.linear1_centre_ppg(ppg)
        out_centre_ecg=self.tanh1_ecg(out_centre_ecg)
        out_centre_ppg=self.tanh1_ecg(out_centre_ppg)
        
        
        out_init=torch.mul(out_centre_ecg,self.w1_centre)+torch.mul(out_centre_ppg,1-self.w1_centre)
        
        out_centre_bn1=self.bn1_centre(out_init)
        out_centre_conv1=self.conv1_centre(out_centre_bn1)
        out_centre_relu1=self.relu1_centre(out_centre_conv1)
        
        out_hidden_1=torch.mul(out_centre_relu1,self.alpha1)+torch.mul((torch.mul(out_ecgsqi_relu1_w,out_ecg_relu1))+(torch.mul(out_ppgsqi_relu1_w,out_ppg_relu1)),(1-self.alpha1))
        
        out_centre_max1=self.max1_centre(out_hidden_1)
        out_centre_drop1=self.drop1_centre(out_centre_max1)
        out_centre_conv2=self.conv2_centre(out_centre_drop1)
        out_centre_max2=self.max2_centre(out_centre_conv2)
        out_centre_relu2=self.relu2_centre(out_centre_max2)
                                                                        
        out_hidden_2=torch.mul(out_centre_relu2,self.alpha2)+torch.mul((torch.mul(out_ecgsqi_relu2_w,out_ecg_relu2))+(torch.mul(out_ppgsqi_relu2_w,out_ppg_relu2)),(1-self.alpha2))
        ################
        out_centre_drop2=self.drop2_centre(out_hidden_2)
        out_centre_conv3=self.conv3_centre(out_centre_drop2)
        out_centre_drop3=self.drop3_centre(out_centre_conv3)
        out_centre_conv4=self.conv4_centre(out_centre_drop3)
        out_centre_max4=self.max4_centre(out_centre_conv4)
        out_centre_relu4=self.relu4_centre(out_centre_max4)
                                                                        
        out_hidden_3=torch.mul(out_centre_relu4,self.alpha3)+torch.mul((torch.mul(out_ecgsqi_relu4_w,out_ecg_relu4))+(torch.mul(out_ppgsqi_relu4_w,out_ppg_relu4)),(1-self.alpha3))                                                                
        
        out_centre_bn2=self.bn2_centre(out_hidden_3)
        out_centre_flatten=self.flat_centre(out_centre_bn2)
                                                                        
        out_hidden_4= torch.mul(out_centre_flatten,self.alpha4)+torch.mul((torch.mul(out_ecgsqi_flat_w,out_ecg_flat))+(torch.mul(out_ppgsqi_flat_w,out_ppg_flat)),(1-self.alpha4))                                                                
                                                                       
        out_centre_drop4=self.drop4_centre(out_hidden_4)
        out_centre_linear=self.linear1_centre(out_centre_drop4)
        out_centre_softm=self.softm_centre(out_centre_linear)

        return out_centre_softm, out_ppg_softm, out_ecg_softm

In [15]:
model_fusion = fusion_model_v3()

In [16]:

ecg_train = torch.from_numpy(ecg_train)
ecg_valid=torch.from_numpy(ecg_valid)
ecg_test=torch.from_numpy(ecg_test)

ppg_train = torch.from_numpy(ppg_train)
ppg_valid=torch.from_numpy(ppg_valid)
ppg_test = torch.from_numpy(ppg_test)

ecg_sqi_train = torch.from_numpy(ecg_sqi_train)
ecg_sqi_valid=torch.from_numpy(ecg_sqi_valid)
ecg_sqi_test = torch.from_numpy(ecg_sqi_test)

ppg_sqi_train = torch.from_numpy(ppg_sqi_train)
ppg_sqi_valid=torch.from_numpy(ppg_sqi_valid)
ppg_sqi_test = torch.from_numpy(ppg_sqi_test)

y_train_wide = torch.from_numpy(y_train_wide)
y_valid_wide = torch.from_numpy(y_valid_wide)
y_test_wide = torch.from_numpy(y_test_wide)
y_train_num = torch.from_numpy(y_train_num)
y_valid_num = torch.from_numpy(y_valid_num)
y_test_num = torch.from_numpy(y_test_num)

In [17]:
if torch.cuda.is_available():
    model_fusion = model_fusion.cuda()
#if torch.cuda.is_available():
#    ecg_valid, y_valid_num, ppg_valid, ecg_sqi_valid, ppg_sqi_valid = ecg_valid.cuda(), y_valid_num.cuda(), ppg_valid.cuda(), ecg_sqi_valid.cuda(), ppg_sqi_valid.cuda()
    #ecg_train, y_train_num, ppg_train, ecg_sqi_train, ppg_sqi_train = ecg_train.cuda(), y_train_num.cuda(), ppg_train.cuda(), ecg_sqi_train.cuda(), ppg_sqi_train.cuda()

In [18]:
# Define the loss function and optimizer
#loss_function = nn.functional.binary_cross_entropy
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_fusion.parameters(), lr=1.5e-5)
batch_size=32
n_batches=np.ceil(len(ecg_train)/batch_size).astype(np.int64)
valid_batch_size=100
valid_batches=np.ceil(len(ecg_valid)/valid_batch_size).astype(np.int64) 

min_valid_loss = np.inf

# Run the training loop
for epoch in range(0, 50): # 5 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0
    current_tot_loss=0.0
    correct=0
    i=0
    # Iterate over the DataLoader for training data
    ind_list = list(range(len(ecg_train)))
    shuffle(ind_list)
    #train_new  = train[ind_list, :,:,:]
    #target_new = target[ind_list,]
    while i<n_batches:

          # Get inputs
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_wide[i*batch_size:(i+1)*batch_size,]
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_num[i*batch_size:(i+1)*batch_size,]
        local_ecg, local_y, local_ppg, local_ecg_sqi,local_ppg_sqi = ecg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),y_train_num[ind_list[i*batch_size:(i+1)*batch_size],].cuda(), ppg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(), ecg_sqi_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),ppg_sqi_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda()
        #if torch.cuda.is_available():
        #    local_X, local_y = local_X.cuda(), local_y.cuda()

          # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs_c, outputs_ecg, outputs_ppg = model_fusion(local_ecg.float(),local_ppg.float(),local_ecg_sqi.float(), local_ppg_sqi.float())
        
        

        # Compute loss
        loss1 = loss_function(outputs_c, local_y)
        #loss2 = loss_function(outputs_ecg, local_y)
        #loss3 = loss_function(outputs_ppg, local_y)
        #loss=(loss1+loss2+loss3)/3
        loss=loss1
        #print(loss)
        
        #loss = loss_function(outputs, local_y.float())
        l2_lambda = 0.001
        l2_reg = torch.tensor(0.).cuda()
        for param in model_fusion.parameters():
            l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()
        
        output = outputs_c.clone()
        output= torch.argmax(output, axis=1)
        
        correct += (output == local_y).float().sum()

        # Print statistics
        
        #current_loss += loss.item()
        current_tot_loss+=loss.item()
       # if i % 20 == 19:
       #     print('Loss after mini-batch %5d: %.3f' %
       #         (i + 1, current_loss / 20))
       #     current_loss = 0.0
        i=i+1
        
    
    
    correct_valid=0
    current_valid_loss = 0.0
    current_valid_tot_loss=0.0
    j=0
    while j<valid_batches:
        model_fusion.eval()     # Optional when not using Model Specific layer
        local_ecg_valid, local_y_valid, local_ppg_valid, local_ecg_sqi_valid,local_ppg_sqi_valid = ecg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),y_valid_num[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(), ppg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(), ecg_sqi_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),ppg_sqi_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda()
        target, target2, target3 = model_fusion(local_ecg_valid.float(),local_ppg_valid.float(),local_ecg_sqi_valid.float(), local_ppg_sqi_valid.float())
        loss1 = loss_function(target,local_y_valid)
        #loss2 = loss_function(target2,local_y_valid)
        #loss3 = loss_function(target3,local_y_valid)
        loss=loss1
        #loss=(loss1+loss2+loss3)/3
        current_valid_loss+=loss.item()
        #valid_loss = loss.item() 
        target= torch.argmax(target, axis=1)
        correct_valid += (target == local_y_valid).float().sum()
        j=j+1
    
    #print(correct)
    #print(current_tot_loss)
    accuracy = 100 * correct / len(ecg_train)
    print(f'Epoch {epoch+1} \t\t Training Loss: {current_tot_loss / n_batches} \t \t Accuracy = {accuracy}, \t Validation Loss: {current_valid_loss/ valid_batches}, \t Validation accuracy: {100*correct_valid/len(ecg_valid)}')
    
    if min_valid_loss > (current_valid_loss/valid_batches):
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{current_valid_loss/valid_batches:.6f}) \t Saving The Model')
        min_valid_loss = (current_valid_loss/valid_batches)
        # Saving State Dict
        torch.save(model_fusion.state_dict(), 'saved_model_fusion_noise_0.01_2_-20.pth')
        
        
    
# Process is complete.
print('Training process has finished.')

Starting epoch 1


D:\conda_envs\torch_19\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:170: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\torch\nn\modules\conv.py:295: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internall

Epoch 1 		 Training Loss: 0.7415015825483584 	 	 Accuracy = 55.79151153564453, 	 Validation Loss: 0.6538472639189826, 	 Validation accuracy: 62.31790542602539
Validation Loss Decreased(inf--->0.653847) 	 Saving The Model
Starting epoch 2
Epoch 2 		 Training Loss: 0.6911732468305898 	 	 Accuracy = 61.3421745300293, 	 Validation Loss: 0.6223917481494926, 	 Validation accuracy: 68.27420043945312
Validation Loss Decreased(0.653847--->0.622392) 	 Saving The Model
Starting epoch 3
Epoch 3 		 Training Loss: 0.6161032403027988 	 	 Accuracy = 73.08454895019531, 	 Validation Loss: 0.5515795716068201, 	 Validation accuracy: 73.86043548583984
Validation Loss Decreased(0.622392--->0.551580) 	 Saving The Model
Starting epoch 4
Epoch 4 		 Training Loss: 0.5957378146721792 	 	 Accuracy = 73.03755187988281, 	 Validation Loss: 0.5376765145892985, 	 Validation accuracy: 74.78853607177734
Validation Loss Decreased(0.551580--->0.537677) 	 Saving The Model
Starting epoch 5
Epoch 5 		 Training Loss: 0.567169

Epoch 38 		 Training Loss: 0.48185174232278777 	 	 Accuracy = 87.24280548095703, 	 Validation Loss: 0.4424210985501607, 	 Validation accuracy: 86.90084838867188
Starting epoch 39
Epoch 39 		 Training Loss: 0.47996485929347965 	 	 Accuracy = 87.40068054199219, 	 Validation Loss: 0.4457903997591364, 	 Validation accuracy: 86.81861114501953
Starting epoch 40
Epoch 40 		 Training Loss: 0.4789702353116713 	 	 Accuracy = 87.51963806152344, 	 Validation Loss: 0.43982692402705814, 	 Validation accuracy: 87.37077331542969
Validation Loss Decreased(0.442290--->0.439827) 	 Saving The Model
Starting epoch 41
Epoch 41 		 Training Loss: 0.47788442611778364 	 	 Accuracy = 87.59527587890625, 	 Validation Loss: 0.43722153041097855, 	 Validation accuracy: 87.55287170410156
Validation Loss Decreased(0.439827--->0.437222) 	 Saving The Model
Starting epoch 42
Epoch 42 		 Training Loss: 0.4746900041398585 	 	 Accuracy = 87.951416015625, 	 Validation Loss: 0.4341245021736413, 	 Validation accuracy: 87.729087

In [20]:
model_saved = fusion_model_v3()
model_saved.load_state_dict(torch.load('saved_model_fusion_noise_0.01_2_-20.pth'))
model_saved.eval()

fusion_model_v3(
  (bn1_ecg): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_ecg): Conv1d(1, 5, kernel_size=(30,), stride=(2,))
  (relu1_ecg): LeakyReLU(negative_slope=0.01)
  (max1_ecg): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1_ecg): Dropout(p=0.25, inplace=False)
  (conv2_ecg): Conv1d(5, 5, kernel_size=(25,), stride=(1,))
  (max2_ecg): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2_ecg): LeakyReLU(negative_slope=0.01)
  (drop2_ecg): Dropout(p=0.25, inplace=False)
  (conv3_ecg): Conv1d(5, 5, kernel_size=(20,), stride=(1,))
  (drop3_ecg): Dropout(p=0.25, inplace=False)
  (conv4_ecg): Conv1d(5, 5, kernel_size=(15,), stride=(1,))
  (max4_ecg): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu4_ecg): LeakyReLU(negative_slope=0.01)
  (bn2_ecg): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (flat_ecg): Fl

In [21]:
model_saved=model_saved.cpu()

In [22]:
outputs_test,out1,out2 = model_saved(ecg_test.float(), ppg_test.float(), ecg_sqi_test.float(), ppg_sqi_test.float())
outputs_test = outputs_test.clone()
outputs_test= torch.argmax(outputs_test, axis=1)

print("centre output")
        
correct = (outputs_test == y_test_num).float().sum()
print(correct/len(ecg_test))

cm=confusion_matrix(y_test_num.cpu(), outputs_test.cpu())
print(cm)

D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:170: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:262: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


centre output
tensor(0.8809)
[[6945 1566]
 [  96 5352]]


## combining losses

In [23]:
model_fusion2 = fusion_model_v3()

In [24]:
if torch.cuda.is_available():
    model_fusion2 = model_fusion2.cuda()

In [25]:
# Define the loss function and optimizer
#loss_function = nn.functional.binary_cross_entropy
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_fusion2.parameters(), lr=2.5e-5)
batch_size=32
n_batches=np.ceil(len(ecg_train)/batch_size).astype(np.int64)
valid_batch_size=100
valid_batches=np.ceil(len(ecg_valid)/valid_batch_size).astype(np.int64) 

min_valid_loss = np.inf

# Run the training loop
for epoch in range(0, 50): # 5 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0
    current_tot_loss=0.0
    correct=0
    i=0
    # Iterate over the DataLoader for training data
    ind_list = list(range(len(ecg_train)))
    shuffle(ind_list)
    #train_new  = train[ind_list, :,:,:]
    #target_new = target[ind_list,]
    while i<n_batches:

          # Get inputs
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_wide[i*batch_size:(i+1)*batch_size,]
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_num[i*batch_size:(i+1)*batch_size,]
        local_ecg, local_y, local_ppg, local_ecg_sqi,local_ppg_sqi = ecg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),y_train_num[ind_list[i*batch_size:(i+1)*batch_size],].cuda(), ppg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(), ecg_sqi_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),ppg_sqi_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda()
        #if torch.cuda.is_available():
        #    local_X, local_y = local_X.cuda(), local_y.cuda()

          # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs_c, outputs_ecg, outputs_ppg = model_fusion2(local_ecg.float(),local_ppg.float(),local_ecg_sqi.float(), local_ppg_sqi.float())
        
        

        # Compute loss
        loss1 = loss_function(outputs_c, local_y)
        loss2 = loss_function(outputs_ecg, local_y)
        loss3 = loss_function(outputs_ppg, local_y)
        loss=(loss1+loss2+loss3)/3
        #loss=loss1
        #print(loss)
        
        #loss = loss_function(outputs, local_y.float())
        l2_lambda = 0.01
        l2_reg = torch.tensor(0.).cuda()
        for param in model_fusion2.parameters():
            l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()
        
        output = outputs_c.clone()
        output= torch.argmax(output, axis=1)
        
        correct += (output == local_y).float().sum()

        # Print statistics
        
        #current_loss += loss.item()
        current_tot_loss+=loss.item()
       # if i % 20 == 19:
       #     print('Loss after mini-batch %5d: %.3f' %
       #         (i + 1, current_loss / 20))
       #     current_loss = 0.0
        i=i+1
        
    
    
    correct_valid=0
    current_valid_loss = 0.0
    current_valid_tot_loss=0.0
    j=0
    while j<valid_batches:
        model_fusion2.eval()     # Optional when not using Model Specific layer
        local_ecg_valid, local_y_valid, local_ppg_valid, local_ecg_sqi_valid,local_ppg_sqi_valid = ecg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),y_valid_num[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(), ppg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(), ecg_sqi_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),ppg_sqi_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda()
        target, target2, target3 = model_fusion2(local_ecg_valid.float(),local_ppg_valid.float(),local_ecg_sqi_valid.float(), local_ppg_sqi_valid.float())
        loss1 = loss_function(target,local_y_valid)
        loss2 = loss_function(target2,local_y_valid)
        loss3 = loss_function(target3,local_y_valid)
        #loss=loss1
        loss=(loss1+loss2+loss3)/3
        current_valid_loss+=loss.item()
        #valid_loss = loss.item() 
        target= torch.argmax(target, axis=1)
        correct_valid += (target == local_y_valid).float().sum()
        j=j+1
    
    #print(correct)
    #print(current_tot_loss)
    accuracy = 100 * correct / len(ecg_train)
    print(f'Epoch {epoch+1} \t\t Training Loss: {current_tot_loss / n_batches} \t \t Accuracy = {accuracy}, \t Validation Loss: {current_valid_loss/ valid_batches}, \t Validation accuracy: {100*correct_valid/len(ecg_valid)}')
    
    if min_valid_loss > (current_valid_loss/valid_batches):
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{current_valid_loss/valid_batches:.6f}) \t Saving The Model')
        min_valid_loss = (current_valid_loss/valid_batches)
        # Saving State Dict
        torch.save(model_fusion2.state_dict(), 'saved_model_fusion_noise_0.01_avg_loss_-20.pth')
        
        
    
# Process is complete.
print('Training process has finished.')

Starting epoch 1


D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:170: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:262: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 		 Training Loss: 1.1502048708918624 	 	 Accuracy = 55.41554641723633, 	 Validation Loss: 0.6418115907942342, 	 Validation accuracy: 56.91376876831055
Validation Loss Decreased(inf--->0.641812) 	 Saving The Model
Starting epoch 2
Epoch 2 		 Training Loss: 0.9757166264676734 	 	 Accuracy = 78.40316772460938, 	 Validation Loss: 0.5059153761780053, 	 Validation accuracy: 82.99459838867188
Validation Loss Decreased(0.641812--->0.505915) 	 Saving The Model
Starting epoch 3
Epoch 3 		 Training Loss: 0.8926639360816855 	 	 Accuracy = 83.24007415771484, 	 Validation Loss: 0.48943646870858487, 	 Validation accuracy: 83.63487243652344
Validation Loss Decreased(0.505915--->0.489436) 	 Saving The Model
Starting epoch 4
Epoch 4 		 Training Loss: 0.8614227798228201 	 	 Accuracy = 84.25636291503906, 	 Validation Loss: 0.4800009087860933, 	 Validation accuracy: 84.92716217041016
Validation Loss Decreased(0.489436--->0.480001) 	 Saving The Model
Starting epoch 5
Epoch 5 		 Training Loss: 0.8422

Epoch 44 		 Training Loss: 0.7124645276821422 	 	 Accuracy = 85.47972106933594, 	 Validation Loss: 0.47426410045540124, 	 Validation accuracy: 86.20183563232422
Starting epoch 45
Epoch 45 		 Training Loss: 0.7118819086651381 	 	 Accuracy = 85.578125, 	 Validation Loss: 0.4746661520840829, 	 Validation accuracy: 86.00211334228516
Starting epoch 46
Epoch 46 		 Training Loss: 0.7109522911811009 	 	 Accuracy = 85.8255844116211, 	 Validation Loss: 0.4749794551852154, 	 Validation accuracy: 85.90225982666016
Starting epoch 47
Epoch 47 		 Training Loss: 0.7095366735151387 	 	 Accuracy = 86.2984848022461, 	 Validation Loss: 0.473384239868811, 	 Validation accuracy: 86.50141143798828
Starting epoch 48
Epoch 48 		 Training Loss: 0.7092230151078307 	 	 Accuracy = 86.21990966796875, 	 Validation Loss: 0.4737964148758448, 	 Validation accuracy: 86.56015014648438
Starting epoch 49
Epoch 49 		 Training Loss: 0.7086130321670071 	 	 Accuracy = 86.05762481689453, 	 Validation Loss: 0.47453308192610044, 

In [26]:
model_saved = fusion_model_v3()
model_saved.load_state_dict(torch.load('saved_model_fusion_noise_0.01_avg_loss_-20.pth'))
model_saved.eval()
model_saved=model_saved.cpu()

In [27]:
outputs_test,out1,out2 = model_saved(ecg_test.float(), ppg_test.float(), ecg_sqi_test.float(), ppg_sqi_test.float())
outputs_test = outputs_test.clone()
outputs_test= torch.argmax(outputs_test, axis=1)
        
correct = (outputs_test == y_test_num).float().sum()
print(correct/len(ecg_test))

cm=confusion_matrix(y_test_num.cpu(), outputs_test.cpu())
print(cm)

D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:149: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:170: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:262: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.8894)
[[7273 1238]
 [ 306 5142]]
